In [83]:
import os, glob, platform, datetime, random
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.autograd import Variable
from torch import functional as F

import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
from PIL import Image
from tensorboardX import SummaryWriter

import numpy as np
from numpy.linalg import inv as denseinv
from scipy import sparse
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv as spinv
import scipy.misc

# Configurations

In [84]:
class Args(object):
    pass
args = Args()
args.test_scene = 'alley_1'
args.arch = "densenet121"
args.epoches = 500
args.epoches_unary_threshold = 0
args.image_h = 256
args.image_w = 256
args.img_extentions = ["png"]
args.training_thresholds = [250,200,150,50,0,300]
args.base_lr = 1
args.lr = args.base_lr
args.snapshot_interval = 5000
args.debug = True
args.gpu_num = 1
args.display_interval = 50
args.display_curindex = 0

system_ = platform.system()
system_dist, system_version, _ = platform.dist()
if system_ == "Darwin": 
    args.train_dir = '/Volumes/Transcend/dataset/sintel2'
    args.pretrained = False
elif platform.dist() ==  ('debian', 'jessie/sid', ''):
    args.train_dir = '/home/albertxavier/dataset/sintel2'
    args.pretrained = True
elif platform.dist() == ('debian', 'stretch/sid', ''):
    args.train_dir = '/home/cad/lwp/workspace/dataset/sintel2'
    args.pretrained = True

if platform.system() == 'Linux': use_gpu = True
else: use_gpu = False

if use_gpu:
    torch.cuda.set_device(args.gpu_num)
    

print(platform.dist())

('debian', 'stretch/sid', '')


# Little Test

In [85]:
# class Netxxx(nn.Module):
#     def build_blocks(self, num_block, num_init_features):
#         bn_size = 4
#         growth_rate = 32
#         drop_rate = 0
#         num_features = num_init_features
#         features = nn.Sequential()
#         for i, num_layers in enumerate(num_block):
#             block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
#                                 bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
#             features.add_module('denseblock%d' % (i + 1), block)
#             num_features = num_features + num_layers * growth_rate
            
#             trans = _MyTransition(num_input_features=num_features, num_output_features=num_features // 2)
#             features.add_module('transition%d' % (i + 1), trans)
#             num_features = num_features // 2
#         print(num_features)
#         return features

#     def __init__(self):
#         super(Netxxx, self).__init__()
#         self.bn = nn.Conv2d(kernel_size=3, in_channels=1, out_channels=1, padding=1)
# #         self.bn = nn.BatchNorm2d(1)
# #         self.fns = nn.Sequential(self.build_blocks([3,3],1),self.build_blocks([3,3],88))
# #         self.fns.add_module(self.build_blocks([3,3],64))
# #         self.fns.add_module(self.build_blocks([3,3],96))
#     def forward(self,x):
#         x = self.bn(x)
# #         x = self.fns(x)
#         return x
    
# netxxx = Netxxx()
# netxxx.cuda()

# cre = nn.MSELoss().cuda()

# optimizer = optim.SGD(net.parameters(), lr=1, momentum=0)


# x = torch.randn(1,1,3,3)
# g = torch.randn(1,1,3,3)
# x = Variable(x.cuda())
# g = Variable(g.cuda())
# y = netxxx(x)
# optimizer.zero_grad()
# loss = cre(y,g)
# loss.backward()
# print(netxxx.bn.weight.grad)

# optimizer.zero_grad()
# y = netxxx(x)
# optimizer.zero_grad()
# loss = cre(y,g)
# loss.backward()
# print(netxxx.bn.weight.grad)

# optimizer.zero_grad()
# y = netxxx(x)
# optimizer.zero_grad()
# loss = cre(y,g)
# loss.backward()
# print(netxxx.bn.weight.grad)

# optimizer.zero_grad()
# y = netxxx(x)
# optimizer.zero_grad()
# loss = cre(y,g)
# loss.backward()
# print(netxxx.bn.weight.grad)

# My DataLoader

In [86]:
def default_loader(path):
    return Image.open(path).convert('RGB')

def make_dataset(dir, phase):
    images_paths = glob.glob(os.path.join(dir, 'clean', '*', '*.png'))
    albedo_paths = images_paths[:]
    shading_paths = images_paths[:]
    pathes = []
    for img_path in images_paths:
        sp = img_path.split('/'); 
        if phase == 'train':
            if sp[-2] == args.test_scene: continue
        else:
            if sp[-2] != args.test_scene: continue
            
        sp[-3] = 'albedo'; 
        sp = ['/'] + sp; 
        albedo_path = os.path.join(*sp)
        
        sp = img_path.split('/'); 
        sp[-3] = 'shading'; 
        sp[-1] = sp[-1].replace('frame', 'out')
        sp = ['/'] + sp; 
        shading_path = os.path.join(*sp)
        
        pathes.append((img_path, albedo_path, shading_path))
    return pathes

class MyImageFolder(data_utils.Dataset):
    def __init__(self, root, phase='train', transform=None, target_transform=None, random_crop=True,
                loader=default_loader):
        imgs = make_dataset(root, phase)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(args.img_extentions)))

        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.random_crop = random_crop
        
    def __getitem__(self, index):
        img_path, albedo_path, shading_path = self.imgs[index]
        
        img = self.loader(img_path)
        albedo = self.loader(albedo_path)
        shading = self.loader(shading_path)
        
        if self.random_crop == True:
            i, j, h, w = self.get_params(img, (int(args.image_h), int(args.image_w)))
            img = img.crop((j, i, j + w, i + h))
            albedo = albedo.crop((j, i, j + w, i + h))
            shading = shading.crop((j, i, j + w, i + h))
#         print(img.size)
#         print((i, j, h, w))
        
        if self.transform is not None: img = self.transform(img)
        if self.transform is not None: albedo = self.transform(albedo)
        if self.transform is not None: shading = self.transform(shading)
        
        scene = img_path.split('/')[-2]
        return img, albedo, shading, scene, img_path
    
    def __len__(self):
        return len(self.imgs)
    
    def get_params(self, img, output_size):
        """Get parameters for ``crop`` for a random crop.
        Args:
            img (PIL Image): Image to be cropped.
            output_size (tuple): Expected output size of the crop.
        Returns:
            tuple: params (i, j, h, w) to be passed to ``crop`` for random crop.
        """
        w, h = img.size
        th, tw = output_size
        if w == tw and h == th:
            return 0, 0, h, w

        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw

    
#             transforms.RandomCrop((args.image_h, args.image_w)),
    
train_dataset = MyImageFolder(args.train_dir, 'train',
                       transforms.Compose(
        [transforms.ToTensor()]
    ), random_crop=True)
test_dataset = MyImageFolder(args.train_dir, 'test', 
                       transforms.Compose(
        [transforms.CenterCrop((args.image_h, args.image_w)),
         transforms.ToTensor()]
    ), random_crop=False)

train_loader = data_utils.DataLoader(train_dataset,1,True,num_workers=1)
test_loader = data_utils.DataLoader(test_dataset,1,True,num_workers=1)

# Load Pretrained Model

[Defination](https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py)
* DenseNet-121: num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16)
    * First Convolution: 32M -> 16M -> 8M
    * every transition: 8M -> 4M -> 2M (downsample 1/2, except the last block)

In [87]:
densenet = models.__dict__[args.arch](pretrained=args.pretrained)

for param in densenet.parameters():
    param.requires_grad = False

if use_gpu:
    densenet.cuda()


In [88]:
# for k,v in enumerate(densenet.parameters()):
#     print (k,v)

In [89]:
class PreTrainedModel(nn.Module):
    def __init__(self, pretrained):
        super(PreTrainedModel, self).__init__()
        common_features_net = nn.Sequential(*list(pretrained.children())[0:1])
        self.net_16M = nn.Sequential(OrderedDict([
            ('conv0', common_features_net[0].conv0),
            ('norm0', common_features_net[0].norm0),
            ('relu0', common_features_net[0].relu0)
        ]))
        self.net_8M = nn.Sequential(OrderedDict([
            ('pool0', common_features_net[0].pool0)
        ]))
        self.net_4M = nn.Sequential(OrderedDict([
            ('denseblock1', common_features_net[0].denseblock1),
            ('transition1', common_features_net[0].transition1)
        ]))
        self.net_2M = nn.Sequential(OrderedDict([
            ('denseblock2', common_features_net[0].denseblock2),
            ('transition2', common_features_net[0].transition2)
        ]))
        self.net_1M = nn.Sequential(OrderedDict([
            ('denseblock3', common_features_net[0].denseblock3),
            ('transition3', common_features_net[0].transition3),
            ('denseblock4', common_features_net[0].denseblock4)
        ]))
    def forward(self, ft_32M):
        
        pretrained_features = [0]*5
        pretrained_features[0] = self.net_16M(ft_32M)
        pretrained_features[1]  = self.net_8M(pretrained_features[0])
        pretrained_features[2]  = self.net_4M(pretrained_features[1])
        pretrained_features[3]  = self.net_2M(pretrained_features[2])
        pretrained_features[4]  = self.net_1M(pretrained_features[3])
        return pretrained_features

if args.debug == True:
#     pass
    print(densenet)
#     common_features_net = nn.Sequential(*list(densenet.children())[0:1])
#     net_x = nn.Sequential(OrderedDict([
#             ('conv0', common_features_net[0].conv0),
#     ]))
    
#     """
#         debug: copy/clone
# #     """
# #     print net_x
#     t = nn.Sequential(*list(net_x.children()))

#     for param in _8M.parameters():
#         print param.data

#     for param in t.parameters():
#         param.data = (param.data*2)
#     print "@@@@@@"

#     for param in t.parameters():
#         print param.data


DenseNet (
  (features): Sequential (
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU (inplace)
    (pool0): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (denseblock1): _DenseBlock (
      (denselayer1): _DenseLayer (
        (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU (inplace)
        (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer (
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [90]:

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
#         self.add_module('relu1', nn.Sigmoid()),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
#         self.add_module('relu2', nn.Sigmoid()),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

class _MyTransition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_MyTransition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
#         self.add_module('relu', nn.Sigmoid())
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=3, stride=1, padding=1))


    
class GradientNet(nn.Module):
    def build_blocks(self, num_block, num_init_features):
        bn_size = 4
        growth_rate = 32
        drop_rate = 0
        num_features = num_init_features
        features = nn.Sequential()
        for i, num_layers in enumerate(num_block):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            features.add_module('mydenseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            
            trans = _MyTransition(num_input_features=num_features, num_output_features=num_features // 2)
            features.add_module('mytransition%d' % (i + 1), trans)
            num_features = num_features // 2
#         return features.cuda()
        return features
    
    def __init__(self, pretrained_model):
        super(GradientNet, self).__init__()
        self.block_config = [(3,3,3),(6,6,6),(12,12,12),(16,16,16),(24,24,24)]
        self.num_input_features = [64,64,128,256,1024]
        self.upsample_config = [2*2,4*2,8*2,16*2,32*2]
        
        self.pretrained_model = pretrained_model
        
#         self.denseblocks = [nn.Sequential()] * len(self.block_config)
#         for i in range(0, len(self.block_config)):
        i=0; self.denseblock16 = self.build_blocks(self.block_config[i], self.num_input_features[i])
        i=1; self.denseblock08 = self.build_blocks(self.block_config[i], self.num_input_features[i])
        i=2; self.denseblock04 = self.build_blocks(self.block_config[i], self.num_input_features[i])
        i=3; self.denseblock02 = self.build_blocks(self.block_config[i], self.num_input_features[i])
        i=4; self.denseblock01 = self.build_blocks(self.block_config[i], self.num_input_features[i])
        
        if use_gpu: self.denseblock16.cuda()
        if use_gpu: self.denseblock08.cuda()
        if use_gpu: self.denseblock04.cuda()
        if use_gpu: self.denseblock02.cuda()
        if use_gpu: self.denseblock01.cuda()
            
        self.num_upsample_input_features = [92,176,352,480,800]
#         for i in range(0, len(self.block_config)):

        i=0; self.upsample16 = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], out_channels=3, kernel_size=self.upsample_config[i], stride=2, padding=1, output_padding=0, groups=1, bias=True, dilation=1)
        i=1; self.upsample08 = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], out_channels=3, kernel_size=self.upsample_config[i], stride=4, padding=2, output_padding=0, groups=1, bias=True, dilation=1)
        i=2; self.upsample04 = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], out_channels=3, kernel_size=self.upsample_config[i], stride=8, padding=4, output_padding=0, groups=1, bias=True, dilation=1)
        i=3; self.upsample02 = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], out_channels=3, kernel_size=self.upsample_config[i], stride=16, padding=8, output_padding=0, groups=1, bias=True, dilation=1)
        i=4; self.upsample01 = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], out_channels=3, kernel_size=self.upsample_config[i], stride=32, padding=16, output_padding=0, groups=1, bias=True, dilation=1)

        if use_gpu: self.upsample16.cuda()
        if use_gpu: self.upsample08.cuda()
        if use_gpu: self.upsample04.cuda()
        if use_gpu: self.upsample02.cuda()
        if use_gpu: self.upsample01.cuda()
        
#             if platform.system() == 'Linux': 
#                 self.upsamples[i] = self.upsamples[i].cuda()
#             stride = stride * 2
        
        self.merge = nn.Sequential()
        self.merge_in_channels =  (3*len(self.block_config), 64, 32, 16)
        self.merge_out_channels = (                      64, 32, 16,  3)
        for i in range(0, len(self.merge_out_channels)): 
            self.merge.add_module('merge.conv{}'.format(i), 
                                  nn.Conv2d(in_channels=self.merge_in_channels[i], 
                                            out_channels=self.merge_out_channels[i], kernel_size=1))
            self.merge.add_module('merge.relu{}'.format(i), nn.ReLU(inplace=True))
                                    
    def forward(self, ft_input):
        ft_pretrained = self.pretrained_model(ft_input)

#         ft_predict = [b(ft_pretrained[i]) for i, b in enumerate(self.denseblocks)]
#         ft_upsampled = [up(ft_predict[i]) for i, up in enumerate(self.upsamples)]
        
        ft_predict   = [0]*len(ft_pretrained)
        ft_upsampled = [0]*len(ft_pretrained)
        
        i = 0; ft_predict[i] = self.denseblock16(ft_pretrained[i])
        i = 1; ft_predict[i] = self.denseblock08(ft_pretrained[i])
        i = 2; ft_predict[i] = self.denseblock04(ft_pretrained[i])
        i = 3; ft_predict[i] = self.denseblock02(ft_pretrained[i])
        i = 4; ft_predict[i] = self.denseblock01(ft_pretrained[i])
        
        i = 0; ft_upsampled[i] = self.upsample16(ft_predict[i])
        i = 1; ft_upsampled[i] = self.upsample08(ft_predict[i])
        i = 2; ft_upsampled[i] = self.upsample04(ft_predict[i])
        i = 3; ft_upsampled[i] = self.upsample02(ft_predict[i])
        i = 4; ft_upsampled[i] = self.upsample01(ft_predict[i])
        
        ft_concated = torch.cat(ft_upsampled, 1)
        ft_merged = self.merge(ft_concated)
#         ft_merged = None
        ft_output = ft_upsampled + [ft_merged]
        return ft_output


In [ ]:
# training loop

ss = 20

args.display_curindex = 0
args.base_lr = 0.05
args.display_interval = 10
args.momentum = 0.9
args.epoches = 500
args.training_thresholds = [ss*4,ss*3,ss*2,ss*1,ss*0,ss*5]
args.power = 0.5

writer = SummaryWriter()

pretrained = PreTrainedModel(densenet)
if use_gpu: 
    pretrained.cuda()
    
net = GradientNet(pretrained)
if use_gpu: 
    net.cuda()

if use_gpu: 
    mse_losses = [nn.MSELoss().cuda()] * 6
    test_losses = [nn.MSELoss().cuda()] * 6
else:
    mse_losses = [nn.MSELoss()] * 6
    test_losses = [nn.MSELoss()] * 6

parameters = filter(lambda p: p.requires_grad, net.parameters())
optimizer = optim.SGD(parameters, lr=args.base_lr, momentum=args.momentum)

def adjust_learning_rate(optimizer, epoch, beg, end, reset_lr=None):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    for param_group in optimizer.param_groups:
        if reset_lr != None:
            param_group['lr'] = reset_lr
            continue
        if epoch != 0: 
            # linear
#             param_group['lr'] *= (end-epoch) / (end-beg)
#             poly base_lr (1 - iter/max_iter) ^ (power)
            param_group['lr'] = args.base_lr * (1. - 1.*epoch/(end-beg)) ** (args.power)
            if param_group['lr'] < 1.0e-8: param_group['lr'] = 1.0e-8
        print('lr', param_group['lr'])
        
# def findLargerInd(target, arr):
#     res = list(filter(lambda x: x>target, arr))
#     print('res',res)
#     if len(res) == 0: return -1
#     return res[0]

for epoch in range(args.epoches):
    net.train()
    print('epoch: {} [{}]'.format(epoch, datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
    
    if epoch < args.training_thresholds[-1]: adjust_learning_rate(optimizer, epoch%20, beg=0, end=20)
    else: adjust_learning_rate(optimizer, epoch, beg=args.training_thresholds[-1], end=args.epoches)
    
    run_losses = [0] * len(args.training_thresholds)
    run_cnts   = [0.00001] * len(args.training_thresholds)
    if (epoch in args.training_thresholds) == True: 
        adjust_learning_rate(optimizer, epoch, reset_lr=args.base_lr, beg=-1, end=-1)
    writer.add_scalar('learning rate', optimizer.param_groups[0]['lr'], global_step=epoch)
    for ind, data in enumerate(train_loader, 0):
        input_img, gt_albedo, gt_shading, test_scene, img_path = data
        im = input_img[0,:,:,:].numpy(); im = im.transpose(1,2,0); im = im[:,:,::-1]*255
        
        if test_scene[0] == 'alley_1':
            print('alley_1 yes')
        input_img = Variable(input_img)
        gt_albedo = Variable(gt_albedo)
        gt_shading = Variable(gt_shading)
        if platform.system() == 'Linux':
            input_img = input_img.cuda()
            gt_albedo = gt_albedo.cuda()
            gt_shading = gt_shading.cuda()
#         run_losses = [0] * len(mse_losses)
#         run_cnts = [0.00001] * len(mse_losses)
        if args.display_curindex % args.display_interval == 0:
            cv2.imwrite('snapshot/input.png', im)

        optimizer.zero_grad()
        ft_predict = net(input_img)
        for (i,threshold) in enumerate(args.training_thresholds):
            if epoch >= threshold:
                if i == 5: s = 1
                else: s = (2**(i+1))
                gt = gt_albedo.cpu().data.numpy()
                n,c,h,w = gt.shape
                gt = gt[0,:,:,:]
                gt = gt.transpose((1,2,0))
                gt = cv2.resize(gt, (h//s, w//s))
                gt = cv2.resize(gt, (h,w))
                if args.display_curindex % args.display_interval == 0:
#                     cv2.imwrite('snapshot/input.png'.format(epoch, i), im)
                    cv2.imwrite('snapshot/gt-{}-{}.png'.format(epoch, i), gt[:,:,::-1]*255)
                gt = gt.transpose((2,0,1))
                gt = gt[np.newaxis, :]
                gt = Variable(torch.from_numpy(gt))
                if use_gpu: gt = gt.cuda()
                loss = mse_losses[i](ft_predict[i], gt)
                loss_data = loss.data.cpu().numpy()
                writer.add_scalar('{}th train iters loss'.format(i), loss_data, global_step=args.display_curindex)
                ma_ = ft_predict[i].max().cpu().data.numpy()
                mi_ = ft_predict[i].min().cpu().data.numpy()
#                 print('mi', mi_, 'ma', ma_)
                writer.add_scalars('{}th train predict'.format(i), {'max': ma_, 'min': mi_}, global_step=args.display_curindex)
#                 run_cnts[i] += 1
                run_losses[i] += loss.data.cpu().numpy()[0]
                loss.backward(retain_graph=True)
                run_cnts[i] += 1
#                 print('i = ', i, '; weig\n', net.upsample01.weight[0,0,0:4,0:4].data.cpu().numpy())
#                 print('i = ', i, '; grad\n', net.upsample01.weight.grad[0,0,0:4,0:4].data.cpu().numpy())
                if args.display_curindex % args.display_interval == 0:
                    im = ft_predict[i].cpu().data.numpy()[0].transpose((1,2,0)) * 255
                    cv2.imwrite('snapshot/train-{}-{}.png'.format(epoch, i), im[:,:,::-1])
        
        optimizer.step()
        args.display_curindex += 1

    """ every epoch """
#     loss_output = 'ind: ' + str(args.display_curindex)
    loss_output = ''
    for i,v in enumerate(run_losses):
        writer.add_scalar('{}th train loss'.format(i), run_losses[i]/ run_cnts[i], global_step=epoch)  
        if i == len(run_losses)-1: 
            loss_output += ' merged: %6f' % (run_losses[i] / run_cnts[i])
            continue
        loss_output += ' %2dM: %6f' % ((2**(4-i)), (run_losses[i] / run_cnts[i]))
    print(loss_output)
    # save at every epoch
    if (epoch+1) % 20 == 0:
        torch.save({
            'epoch': epoch,
            'args' : args,
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict()
        }, 'snapshot/snapshot-{}.pth.tar'.format(epoch))
    
    # test 
    net.eval()
    test_losses = [0] * len(args.training_thresholds)
    test_cnts   = [0.00001] * len(args.training_thresholds)   
    for ind, data in enumerate(test_loader, 0):
        input_img, gt_albedo, gt_shading, test_scene, img_path = data
        input_img = Variable(input_img)
        gt_albedo = Variable(gt_albedo)
        gt_shading = Variable(gt_shading)
        if use_gpu:
            input_img = input_img.cuda(args.gpu_num)
        ft_test = net(input_img)
            
        for i,v in enumerate(ft_test):
            if epoch < args.training_thresholds[i]: continue
            if i == 5: s = 1
            else: s = (2**(i+1))
            gt = gt_albedo.data.numpy()
            n,c,h,w = gt.shape
            gt = gt[0,:,:,:]
            gt = gt.transpose((1,2,0))
            gt = cv2.resize(gt, (h//s, w//s))
            gt = cv2.resize(gt, (h,w))
            
            gt = gt.transpose((2,0,1))
            gt = gt[np.newaxis, :]
            gt = Variable(torch.from_numpy(gt))
            if use_gpu: gt = gt.cuda()

            loss = mse_losses[i](ft_test[i], gt)
            
            test_losses[i] += loss.data.cpu().numpy()[0]
            test_cnts[i] += 1
#             v = v[0].cpu().data.numpy()
#             v = v.transpose(1,2,0)
#             if i == 0: cv2.imwrite('snapshot/test-{}-{}.png'.format(epoch, i), v[:,:,::-1]*255)
    
    for i,v in enumerate(test_losses):
        writer.add_scalar('{}th test loss'.format(i), test_losses[i]/test_cnts[i], global_step=epoch)

epoch: 0 [2017-11-13 21:30]
lr 0.05
 16M: 0.000000  8M: 0.000000  4M: 0.000000  2M: 0.000000  1M: 0.044705 merged: 0.000000
epoch: 1 [2017-11-13 21:32]
lr 0.04873397172404482
 16M: 0.000000  8M: 0.000000  4M: 0.000000  2M: 0.000000  1M: 0.042118 merged: 0.000000
epoch: 2 [2017-11-13 21:34]
lr 0.047434164902525694
 16M: 0.000000  8M: 0.000000  4M: 0.000000  2M: 0.000000  1M: 0.039526 merged: 0.000000
epoch: 3 [2017-11-13 21:36]
lr 0.04609772228646444


In [ ]:
# net.upsample01.weight.grad[0,0,0:4,0:4].data[0]
for k,v in enumerate(net.parameters()):
    print (k,v)

In [54]:
for k,v in enumerate(test_loader, 0):
    a,b,c,_,_=v
    print(a[0,0,0:4,0:4])
    break
for k,v in enumerate(test_loader, 0):
    a,b,c,_,_=v
    print(a[0,0,0:4,0:4])
    break


 0.1294  0.1373  0.1373  0.1490
 0.1333  0.1373  0.1333  0.1490
 0.1333  0.1373  0.1294  0.1529
 0.1294  0.1373  0.1255  0.1490
[torch.FloatTensor of size 4x4]


 0.4549  0.3137  0.2196  0.2157
 0.4196  0.3020  0.2392  0.2157
 0.3882  0.2706  0.2471  0.2353
 0.3451  0.2510  0.2314  0.2471
[torch.FloatTensor of size 4x4]



# Visualize Graph

In [161]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph

    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.creator)
    return dot

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
make